In [99]:
#Dependecies 
import pandas as pd
import numpy as np

In [100]:
#Open CSV & read 

purchase_data = pd.read_csv("purchase_data.csv")



purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [101]:
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [102]:
total_players = purchase_data ['SN'].nunique()
total_players_df = pd.DataFrame({'Total Players': [total_players]})
total_players_df

,Total Players
0,576


In [103]:
unique_items = len(purchase_data['Item ID'].unique())
average_price = purchase_data ['Price'].mean()
num_purchases = purchase_data ['Purchase ID'].count()
revenue = purchase_data ['Price'].sum()

summary_table = pd.DataFrame({'Number of Unique Items': [unique_items],
                              'Average Price': ['${:,.2f}'.format(average_price)],
                              'Number of Purchases': [num_purchases],
                              'Total Revenue': ['${:,.2f}'.format(revenue)]})
summary_table



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [104]:
#Gender
genders = purchase_data["Gender"].unique()
genders

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [105]:
male_players = purchase_data.loc[purchase_data['Gender']== 'Male','SN'].nunique()
female_players = purchase_data.loc[purchase_data['Gender']== 'Female','SN'].nunique()
other_players = purchase_data.loc[purchase_data['Gender']== 'Other / Non-Disclosed','SN'].nunique()

male_p = "{:.2%}".format(male_players/purchase_data['SN'].nunique())
female_p = "{:.2%}".format(female_players/purchase_data['SN'].nunique())
other_p = "{:.2%}".format(other_players/purchase_data['SN'].nunique())

gender_summary_table = pd.DataFrame({'':['Male', 'Female', 'Other / Non-Disclosed'],
                                     'Total Count':[male_players,female_players,other_players],
                                     'Percentage of Players':[male_p, female_p, other_p]})
gender_summary_table = gender_summary_table.set_index('')
gender_summary_table


,Total Count,Percentage of Players
,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [106]:
#Purchasing Analysis (Gender)
gender_df = purchase_data.groupby(["Gender"])

purchase_count = gender_df["Price"].count()
price_mean = gender_df["Price"].mean()
total_purchase = gender_df["Price"].sum()
nrml_price_mean = gender_df["Price"].sum() / gender_df["Price"].count()

gender_purchasing = pd.DataFrame({"Purchase Count": purchase_count,
                                  "Average Purchase Price": price_mean,
                                  "Total Purchase Value": total_purchase,
                                  "Normalized Totals" : nrml_price_mean
                                   })

gender_purchasing["Average Purchase Price"] = gender_purchasing["Average Purchase Price"].astype(float).map("${:,.2f}".format)
gender_purchasing["Total Purchase Value"] = gender_purchasing["Total Purchase Value"].astype(float).map("${:,.2f}".format)
gender_purchasing["Normalized Totals"] = gender_purchasing["Normalized Totals"].astype(float).map("${:,.2f}".format)

gender_purchasing

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,"$1,967.64",$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


In [120]:

#Step 5. The age demographics purchasing analysis, each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.):
#For this analysis I'll create a new df of user name, price, and age:
sn_age_df = purchase_data[["SN", "Age","Price"]]

#Now, we'll find the total number of players per age range... using binning:
age_bin_list = [0,9,14,19,24,29,34,39,200]
age_range_list = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

sn_age_df['Age Range'] = pd.cut(sn_age_df['Age'], age_bin_list, labels= age_range_list)
uni_sn_age_df = sn_age_df.drop_duplicates(subset='SN',keep='first', inplace=False)

counts_of_age_totals = uni_sn_age_df['Age Range'].value_counts()
age_range_percentage = counts_of_age_totals.divide(other=total_players)
age_range_percentage = round(age_range_percentage.multiply(other=100),2)

age_demographics_df = pd.DataFrame({
    "Total Count" : counts_of_age_totals,
    "Percentage of Players" : age_range_percentage
})


age_demographics_df.sort_index()

<ipython-input-120-b5d315c8cc73>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sn_age_df['Age Range'] = pd.cut(sn_age_df['Age'], age_bin_list, labels= age_range_list)


,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [121]:
#Next we can organize the data by age range and compute a financial analysis:

grouped_ages = sn_and_age_df.groupby('Age Range')

#a. Purchase Count
age_grouped_purch_count = grouped_ages['Price'].count()
#b. Average Purchase Price
age_grouped_avg = round(grouped_ages['Price'].mean(),2)
#c. Total Purchase Value
age_grouped_tot_purch_sum = grouped_ages['Price'].sum()
#d. Average Purchase Total per Person by Age Group
age_grouped_avg_tot_per_person = round(age_grouped_tot_purch_sum/counts_of_age_totals,2)

age_purch_summary_df = pd.DataFrame({
    "Purchase Count" : age_grouped_purch_count,
    "Average Purchase Price" : age_grouped_avg,
    "Total Purchase Value" : age_grouped_tot_purch_sum,
    "Average Total Purchase per Person" : age_grouped_avg_tot_per_person
})

age_purch_summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [122]:
#Step 6. Identify top spenders:
#a. First found the top 5 spenders in the game by total purchase value:
sns_and_prices_df = purchase_data[["SN","Price"]]
grouped_sns = sns_and_prices_df.groupby('SN')

#a1. Purchase Count
sn_grouped_purch_count = grouped_sns['Price'].count()

#a2. Total Purchase Value
sn_grouped_tot_purch_sum = grouped_sns['Price'].sum()

#a3. Average Purchase Price
avg_grouped_purch_per_sn = round(sn_grouped_tot_purch_sum/sn_grouped_purch_count,2)

#b. DataFrame of SN, Purchase Count, Average Purchase Price, Total Purchase Value:
sns_and_tot_prices_df = pd.DataFrame({
    "Purchase Count" : sn_grouped_purch_count,
    "Average Purchase Price" : avg_grouped_purch_per_sn,
    "Total Purchase Value" : sn_grouped_tot_purch_sum

})

sns_and_tot_prices_df = sns_and_tot_prices_df.sort_values('Total Purchase Value', ascending=False)

top_5_spenders_df = sns_and_tot_prices_df.head()

top_5_spenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [123]:

#a. First found the top 5 items by purchase count:
itemname_itemid_df= purchase_data[["Item ID","Item Name","Price"]]
grouped_items = itemname_itemid_df.groupby('Item ID')
grouped_item_names = grouped_items["Item Name"].unique()

#a1. Item Purchase Count:
items_grouped_total_purch = grouped_items['Price'].count()

#a2. Total Purchase Value:
items_grouped_tot_purch_sum = grouped_items['Price'].sum()

#a3. Item Price:
items_grouped_price = items_grouped_tot_purch_sum/items_grouped_total_purch

#b. DataFrame of the Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value:

pop_items_summary_df = pd.DataFrame({
    "Item Name" : grouped_item_names,
    "Purchase Count" : items_grouped_total_purch,
    "Item Price" : items_grouped_price,
    "Total Purchase Value" : items_grouped_tot_purch_sum

})
pop_items_summary_df = pop_items_summary_df.sort_values('Purchase Count', ascending=False)

pop_5_items_df = pop_items_summary_df.head()

pop_5_items_df

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,[Final Critic],13,4.614615,59.99
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,4.230000,50.76
145,[Fiery Glass Crusader],9,4.580000,41.22
132,[Persuasion],9,3.221111,28.99
108,"[Extraction, Quickblade Of Trembling Hands]",9,3.530000,31.77


In [124]:

# Find the 5 most profitable items by total purchase value:


top_items_summary_df = pop_items_summary_df.sort_values('Total Purchase Value', ascending=False)

top_5_items_df = top_items_summary_df.head()

top_5_items_df

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,[Final Critic],13,4.614615,59.99
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,4.230000,50.76
82,[Nirvana],9,4.900000,44.10
145,[Fiery Glass Crusader],9,4.580000,41.22
103,[Singed Scalpel],8,4.350000,34.80
